In [4]:
import requests
import pandas as pd
#import datetime
import time

def getData(ticker, metric, no_of_days):
    
    ticker = ticker + '.NS'
    current_time = int(round(time.time()))
    start_time = current_time - (no_of_days)*24*60*60
    
    
    resp = requests.get('https://query1.finance.yahoo.com/v8/finance/chart/'
                        +ticker+'?formatted=true&crumb=3fOAXVz3HoU&lang=en-IN&region=IN&period1='
                        +str(start_time)+'&period2='+str(current_time)
                        +'&interval=1d&events=div%7Csplit&corsDomain=in.finance.yahoo.com')
    
    
    result = resp.json()
    
    
    timestamps = result['chart']['result'][0]['timestamp']
    values = result['chart']['result'][0]['indicators']['quote'][0][metric]
    
    #len(values)
    #datetime.datetime.fromtimestamp(timestamps[len(timestamps) - 1])
    
    
    d = {'timestamp': timestamps, 'value': values}
    dataFrame = pd.DataFrame(data=d)
    
    return dataFrame

In [52]:
val = getData('SBIN', 'close', 350)

In [34]:
val['5SMA'] = val['value'].rolling(window = 25).mean()

In [39]:
window_length = 25
data = val
sma = data['value'].rolling(window = window_length).mean()
d = {'timestamp': data['timestamp'], 'sma': sma}
dataFrame = pd.DataFrame(data=d)

In [42]:
import numpy as np

In [46]:
data['std'] = data['value'].rolling(window = 20).std() * 2
data['sma20'] = data['value'].rolling(window = 20).mean()

In [47]:
data['lower'] = data['sma20'] - data['std']

In [49]:
data.tail()

,timestamp,value,5SMA,std,lower,sma20
231,1548042300,292.450012,297.055996,9.515549,288.359446,297.874995
232,1548128700,290.250000,297.085996,9.889493,287.845503,297.734996
233,1548215100,286.649994,296.841996,10.971469,286.388527,297.359996
234,1548301500,288.049988,296.335995,11.520819,285.634177,297.154996
235,1548387900,285.399994,295.989995,12.638121,284.046875,296.684996


In [57]:
def get_bollinger_bands(data, window_length, variation):
    sma = data['value'].rolling(window = window_length).mean()
    std = data['value'].rolling(window = window_length).std()

    d = {'timestamp': data['timestamp'], 'sma': sma, 'std': std}
    dataFrame = pd.DataFrame(data=d)
    
    dataFrame['upper_band'] = dataFrame['sma'] + variation*dataFrame['std']
    dataFrame['lower_band'] = dataFrame['sma'] - variation*dataFrame['std']
    
    return dataFrame

In [58]:
get_bollinger_bands(val, 20, 2)

,timestamp,sma,std,upper_band,lower_band
0,1518407100,NaN,NaN,NaN,NaN
1,1518579900,NaN,NaN,NaN,NaN
2,1518666300,NaN,NaN,NaN,NaN
3,1518752700,NaN,NaN,NaN,NaN
4,1519011900,NaN,NaN,NaN,NaN
5,1519098300,NaN,NaN,NaN,NaN
6,1519184700,NaN,NaN,NaN,NaN
7,1519271100,NaN,NaN,NaN,NaN
8,1519357500,NaN,NaN,NaN,NaN
9,1519616700,NaN,NaN,NaN,NaN


In [59]:
val.tail()

,timestamp,value
231,1548042300,292.450012
232,1548128700,290.250000
233,1548215100,286.649994
234,1548301500,288.049988
235,1548387900,285.399994
